In [62]:
import bs4
from bs4 import BeautifulSoup
from selenium import webdriver
import requests

import random
import time
import dateparser
import pandas as pd
import math
from urllib import parse
import re
import os

In [2]:
my_headers_Get = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1'
}

def getPage(url):
    delay = 0.1*random.random()
    time.sleep(delay)
    s = requests.Session()
    return s.get(url, headers=my_headers_Get)

def robust_select(soup,SELECTOR):
    if type(SELECTOR) is not str:
        return None
    else:
        res = soup.select(SELECTOR)
        if res is None:
            return None
        else:
            if len(res) == 1:
                return res[0].text
            else:
                rescombined = ''
                for part in res:
                    rescombined += part.text
                return rescombined

In [148]:
city = '长春'

websiteInfo = pd.read_csv('settings/city_govs.csv',index_col='city')

start_urls = websiteInfo['url'][city]
MAIN_PAGE_ENCODING = websiteInfo['main_page_encoding'][city]
SUB_PAGE_ENCODING = websiteInfo['sub_page_encoding'][city]

#Load the selectors for the city
selectorInfo = pd.read_csv('settings/selector_settings.csv',index_col='city')
NEWS_SELECTOR = selectorInfo['news_selector'][city]
URL_SELECTOR = selectorInfo['url_selector'][city]
NEXT_PAGE_SELECTOR = selectorInfo['next_page_selector'][city]


#Test url
mainpageUrl = 'http://www.changchun.gov.cn/zw_33994/tzgg/'
subpageUrl = 'http://www.changchun.gov.cn/zw_33994/tzgg/202004/t20200419_2345730.html'



#options = webdriver.ChromeOptions()
#options.add_argument('headless')

#driver = webdriver.Chrome(options=options)
#driver.get(start_urls)
#soup = BeautifulSoup(driver.page_source, 'html.parser')

In [155]:
NEWS_SELECTOR = 'div.xyh_listul>ul.currency_ul>li>a'

In [ ]:
soup_main

In [156]:
mainpage = getPage(mainpageUrl)
soup_main = BeautifulSoup(mainpage.content.decode(MAIN_PAGE_ENCODING,'ignore'), 'html.parser')
newsList = soup_main.select(NEWS_SELECTOR)
nextPage = soup_main.select(NEXT_PAGE_SELECTOR)

In [157]:
newsList

[<a href="./202005/t20200501_2350563.html" target="_blank" title="长春市人民政府关于开展交通组织调整的通告">长春市人民政府关于开展交通组织调整的通告</a>,
 <a href="./202004/t20200429_2349891.html" target="_blank" title="长春市政务服务中心迁址公告">长春市政务服务中心迁址公告</a>,
 <a href="./202004/t20200429_2349748.html" target="_blank" title="长春市人民政府网站关于临时关闭依申请公开在线申请功能的通知">长春市人民政府网站关于临时关闭依申请公开在线申请功能的通知</a>,
 <a href="./202004/t20200424_2347541.html" target="_blank" title="长春市惠满春城消费季“普惠型”消费券发放公告">长春市惠满春城消费季“普惠型”消费券发放公告</a>,
 <a href="./202004/t20200423_2347199.html" target="_blank" title="2020年“五·一”国际劳动节放假通知">2020年“五·一”国际劳动节放假通知</a>,
 <a href="./202004/t20200419_2345730.html" target="_blank" title="长春市新型冠状病毒肺炎疫情防控工作领导小组办公室公告">长春市新型冠状病毒肺炎疫情防控工作领导小组办公室公告</a>,
 <a href="./202004/t20200410_2298339.html" target="_blank" title="长春市地方志编纂委员会、长春市方志馆面向社会征集抗击新冠肺炎疫情相关资料公告">长春市地方志编纂委员会、长春市方志馆面向社会征集抗击新冠肺炎疫情相关资料公告</a>,
 <a href="./202003/t20200327_2241348.html" target="_blank" title="清明节放假通知">清明节放假通知</a>,
 <a href="./202003/t20200320_2237184.html" target="_blank" t

In [160]:
nextPage

[]

In [119]:
TITLE_SELECTOR = 'div.zx_xxgk_cont>div.tit>h1'
DATE_SELECTOR = selectorInfo['date_selector'][city]
SOURCE_SELECTOR = selectorInfo['source_selector'][city]
DPRT_SELECTOR = selectorInfo['department_selector'][city]
CONTENT_SELECTOR = 'div.news_cont_d_wrap>p'

In [120]:
subpage = getPage(subpageUrl)
soup_sub = BeautifulSoup(subpage.content.decode(SUB_PAGE_ENCODING,'ignore'), 'html.parser')
title = soup_sub.select(TITLE_SELECTOR)
title_final = robust_select(soup_sub,TITLE_SELECTOR)
date = soup_sub.select(DATE_SELECTOR)
date_final = robust_select(soup_sub,DATE_SELECTOR)
content = soup_sub.select(CONTENT_SELECTOR)
content_final = robust_select(soup_sub,CONTENT_SELECTOR)

In [84]:
date_final

' 信息来源：深圳市交通运输局 信息提供日期：2020-04-14  【字体：大中小 】 视力保护色： '

In [122]:
content_final

'\u3000\u3000深圳地铁集团表示，为配合新线开通工作，地铁2号线将于4月17日及18日调整运营服务时间，届时将开行黄贝岭站-赤湾站和黄贝岭站-后海站两条公交接驳线路。\u3000\u30004月17日，地铁2号线将提前结束运营，运营时间调整为6∶30-21∶00；18日，地铁2号线将推迟开站运营时间，运营时间计划调整为9∶00-23∶00。为了方便期间市民乘客的出行，届时将组织循环开行2号线的两条公交接驳线路：公交接驳慢线为黄贝岭站-赤湾站，除新秀、燕南、华强北、深康、红树湾5个站点不停靠之外，其余各站均停靠上下客；公交接驳快线为黄贝岭站-后海站，仅停靠黄贝岭、大剧院、市民中心、福田、景田、安托山、世界之窗、后海共8个换乘站。（来源：深圳特区报）'

In [121]:
content

[<p indenttext="　　" noextractcontent="true" style="text-align: justify;">　　深圳地铁集团表示，为配合新线开通工作，地铁2号线将于4月17日及18日调整运营服务时间，届时将开行黄贝岭站-赤湾站和黄贝岭站-后海站两条公交接驳线路。</p>,
 <p indenttext="　　" noextractcontent="true" style="text-align: justify;">　　4月17日，地铁2号线将提前结束运营，运营时间调整为6∶30-21∶00；18日，地铁2号线将推迟开站运营时间，运营时间计划调整为9∶00-23∶00。为了方便期间市民乘客的出行，届时将组织循环开行2号线的两条公交接驳线路：公交接驳慢线为黄贝岭站-赤湾站，除新秀、燕南、华强北、深康、红树湾5个站点不停靠之外，其余各站均停靠上下客；公交接驳快线为黄贝岭站-后海站，仅停靠黄贝岭、大剧院、市民中心、福田、景田、安托山、世界之窗、后海共8个换乘站。（来源：深圳特区报）</p>]

In [58]:
a = dateparser.parse(date_final)

In [109]:
re.findall('([0-9]+[-年月日/0-9\.]*[0-9]+[日]?)',date_test)

['2020年04月14日', '2012-09-11', '2019/12/11', '2011.01.11']

In [162]:
s = pd.read_csv('settings/selector_settings.csv',index_col='city')

In [141]:
s.loc['深圳'][['title_selector','content_selector']] = [TITLE_SELECTOR,CONTENT_SELECTOR]

In [142]:
s[['title_selector','content_selector']]

,title_selector,content_selector
city,,
广州,h1.content_title,#zoomcon > p
北京,div.header>h1,#mainText
上海,div#ivs_title,div#ivs_content>p
深圳,div.zx_xxgk_cont>div.tit>h1,div.news_cont_d_wrap>p
哈尔滨,td.title>h2.title>script,NaN
南宁,div#ucap-title,#zoom
太原,NaN,NaN
长春,div.xyh_xxynrq>h2,#Zoom
合肥,h1.newstitle,#zoom


In [143]:
s.to_csv('settings/selector_settings.csv',encoding='utf_8_sig')

In [168]:
csvFiles = os.listdir('results/csv/')
for i in csvFiles:
    if i.endswith('csv'):
        print(i)

广州.csv
上海.csv
深圳.csv
